In [1]:
from bs4 import BeautifulSoup as BS
import requests
import csv
import re
import pandas as pd
from pandas import DataFrame

In [2]:
def turnp(i):    #翻页代码
    if i == 0:
        url='https://search.jd.com/Search?keyword=%E7%AC%94%E8%AE%B0%E6%9C%AC%E7%94%B5%E8%84%91%E8%87%AA%E8%90%A5&psort=3&spm=2.1.0&psort=3&click=0%27'
    else:
        url='https://search.jd.com/Search?keyword=%E7%AC%94%E8%AE%B0%E6%9C%AC%E7%94%B5%E8%84%91%E8%87%AA%E8%90%A5&psort=3&spm=2.1.0&psort=3&pvid=ce328885a0b24d17852a8d13f6497e27&page='+str(i+1)+'&s='+str(30*i+1)+'&click=0'
    return url

def gettext(url):  #爬取文本
    try:
        head={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
        r=requests.get(url,headers=head)
        r.raise_for_status()
        r.encoding=r.apparent_encoding
        text=r.text        
        return text
    except:
        return '爬取失败'

def get_price_html(text): #爬取每一页上的价格和url地址,返回列表
    L_price=[]
    L_url=[]
    soup=BS(text,'html.parser')
    price_all=soup.findAll('i')
    url_all=re.findall(r'//item.jd.com/[0-9]+.html',text)
    L=[]
    for price in price_all:
        try:
            f=float(price.string)
            if f>1000:
                L_price.append(f)
        except:
                continue
    for  x_url in url_all:
        if x_url not in L_url:
            L_url.append(x_url)
    #print(len(L_price),len(L_url))
    for i in range(len(L_price)):
        L.append([L_price[i],L_url[i]])
    return L



In [3]:
L=[]
for t in range(10): #遍历10页    
    url=turnp(t)    
    text=gettext(url)
    L1=get_price_html(text)
    L.extend(L1)
len(L)

300

In [4]:
with open('网址.csv','w') as f1: #爬取10页网址与价格，保存为csv文件
    f=csv.writer(f1)
    for i in L:
        f.writerow(i)
    print('over')

over


In [10]:
def getinfo(text): #得到每个url的详细信息
    soup=BS(text,'html.parser')
    m=soup.findAll('li')
    L=[]
    try:
        name=re.search('商品名称：.+?<',str(m))
        L.append(name.group()[5:-1])
    except:
        L.append('none')
    try:   
        wei=re.search('裸机重量：.+?<',str(m))
        L.append(wei.group()[5:-1])
    except:
        L.append('none')
    try:
        thi=re.search('厚度：.+?<',str(m))
        L.append(thi.group()[3:-1])
    except:
        L.append('none')
    try:
        big=re.search('屏幕尺寸：.+?<',str(m))
        L.append(big.group()[5:-1])
    except:
        L.append('none')
    try:
        time=re.search('待机时长：.+?<',str(m))
        L.append(time.group()[5:-1])
    except:
        L.append('none')
    try:
        color=re.search('屏幕色域：.+?<',str(m))
        L.append(color.group()[5:-1])
    except:
        L.append('none')
    try:
        sys=re.search('系统：.+?<',str(m))
        L.append(sys.group()[3:-1])
    except:
        L.append('none')
    try:
        chuliqi=re.search('处理器：.+?<',str(m))
        L.append(chuliqi.group()[4:-1])
    except:
        L.append('none')
    try:
        try:
            neicun=re.search('内存容量：[0-9]+?GB<',str(m))
            L.append(neicun.group()[5:-1])
        except:
            neicun=re.search('内存：[0-9]+?GB<',str(m))
            L.append(neicun.group()[3:-1])    
    except:
        L.append('none')
    try:
        yinpan=re.search('固态硬盘（SSD）：[0-9]+?GB<',str(m))
        L.append(yinpan.group()[10:-1])    
    except:
        L.append('none')
    try:
        try:
            xianka=re.search('显卡(型号|类别)：.+?<',str(m))
            L.append(xianka.group()[5:-1])
        except:
            xianka=re.search('显卡：.+?<',str(m))
            L.append(xianka.group()[3:-1])
    except:
        L.append('none')
    return L

In [11]:
with open('网址.csv') as f1: #将数据输出至csv
    reader=csv.reader(f1)
    i=0
    with open('数据.csv','w',encoding='utf-8') as f2:
        writer=csv.writer(f2)
        writer.writerow(['价格','网址','商品名称','裸机重量','厚度','屏幕尺寸','待机时长','屏幕色域','系统','处理器','内存','固态硬盘','显卡'])
        for row in reader:
            if i%2==0:
                url=row[1]
                url='https:'+url
                text=gettext(url)
                L=getinfo(text)
                L=row+L
                writer.writerow(L)
            i=i+1

    print('over')

over
